# breweryData
### pull brewery data from OpenBreweryDB API

In [1]:
# import dependencies
import pandas as pd
import numpy as np
import requests
from config import gkey

apistub = "https://api.openbrewerydb.org/breweries"

In [2]:
# hit API
combodf = pd.DataFrame()
urlstub = apistub+"?by_state=north%20carolina&page="
for p in range(30):
    url = urlstub + str(p)
    resp = requests.get(url).json()
    df = pd.DataFrame(resp)
    combodf = combodf.append(df,ignore_index=True)

In [3]:
# display dataframe extracted
combodf

,id,name,brewery_type,street,city,state,postal_code,country,longitude,latitude,phone,website_url,updated_at,tag_list
0,5016,Bark Brewing Company,brewpub,3021 Spring Garden St,Greensboro,North Carolina,27403-1968,United States,-79.8436043406271,36.0620103554184,3368561406,http://barkbrewingcompany.com,2018-08-24T15:41:02.410Z,[]
1,5033,Blowing Rock Brewing Company,brewpub,152 Sunset Dr,Blowing Rock,North Carolina,28605-7205,United States,-81.677229,36.1327196,8284149600,http://www.blowingrockbrewing.com,2018-08-24T15:41:10.512Z,[]
2,5035,Blue Ghost Brewing Company,brewpub,125 Underwood Rd,Fletcher,North Carolina,28732-,United States,-82.530580304217,35.4368388,8283760159,http://www.blueghostbrewing.com,2018-08-24T15:41:11.489Z,[]
3,5067,Bull Durham Beer Co,micro,409 Blackwell St,Durham,North Carolina,27701-3972,United States,-78.90481567784,35.9915653,9197443568,http://www.bulldurhambeer.com,2018-08-24T15:41:22.982Z,[]
4,5090,Dirtbag Ales Brewery and Taproom,micro,3623 Legion Rd,Hope Mills,North Carolina,28348-8413,United States,-78.9220969691311,34.9939752576824,9104262537,http://www.dirtbagales.com,2018-08-24T15:41:33.473Z,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,5273,Thirsty Monk Brewery,brewpub,92 Patton Ave,Asheville,North Carolina,28801-3315,United States,-82.5553147520814,35.59410425,8282545470,http://www.monkpub.com,2018-08-24T15:42:51.462Z,[]
331,5283,Triskelion Brewing Company,micro,340 7th Avenue East,Hendersonville,North Carolina,28792,United States,-82.4578969903109,35.3206001746298,8283887051,http://www.triskelionbrewing.com,2018-08-24T15:42:55.346Z,[]
332,5293,Valley River Brewery & Eatery,brewpub,71 Tennessee St,Murphy,North Carolina,28906-2938,United States,-84.0352081706441,35.0889040369639,8288372337,http://www.valleyriverbrewery.com,2018-08-24T15:42:59.685Z,[]
333,5304,White Street Brewing Company,micro,218 S White St,Wake Forest,North Carolina,27587-2742,United States,-78.5100617,35.9774896,9196479439,http://www.whitestreetbrewing.com,2018-08-24T15:43:05.047Z,[]


In [4]:
# subset dataframe on only brewery types we want
subdf = combodf[combodf['brewery_type'].isin(['brewpub','micro','regional','large'])]

# remove tag_list column since it's empty
del subdf['tag_list']

# drop duplicates
subdf = subdf.drop_duplicates()

# extract zip from postal_code
subdf['zipcode'] = [cd[0:5] for cd in subdf['postal_code']]

In [5]:
# fill in lat/lng gaps
base_url = 'https://maps.googleapis.com/maps/api/geocode/json'
for index, row in subdf.iterrows():
    if row.latitude is None:
        address = f"{row.street}, {row.city}, {row.state}"
        params = {
            "address": address,
            "key": gkey
        }

        response = requests.get(base_url, params=params)
        results = response.json()["results"][0]['geometry']['location']

        try:
            subdf.at[index, 'latitude'] = results["lat"]
            subdf.at[index, 'longitude'] = results["lng"]
        except:
            pass
        
    if row.street == '':
        subdf.drop(index, inplace=True)

In [6]:
# display final dataframe
subdf

,id,name,brewery_type,street,city,state,postal_code,country,longitude,latitude,phone,website_url,updated_at,zipcode
0,5016,Bark Brewing Company,brewpub,3021 Spring Garden St,Greensboro,North Carolina,27403-1968,United States,-79.8436043406271,36.0620103554184,3368561406,http://barkbrewingcompany.com,2018-08-24T15:41:02.410Z,27403
1,5033,Blowing Rock Brewing Company,brewpub,152 Sunset Dr,Blowing Rock,North Carolina,28605-7205,United States,-81.677229,36.1327196,8284149600,http://www.blowingrockbrewing.com,2018-08-24T15:41:10.512Z,28605
2,5035,Blue Ghost Brewing Company,brewpub,125 Underwood Rd,Fletcher,North Carolina,28732-,United States,-82.530580304217,35.4368388,8283760159,http://www.blueghostbrewing.com,2018-08-24T15:41:11.489Z,28732
3,5067,Bull Durham Beer Co,micro,409 Blackwell St,Durham,North Carolina,27701-3972,United States,-78.90481567784,35.9915653,9197443568,http://www.bulldurhambeer.com,2018-08-24T15:41:22.982Z,27701
4,5090,Dirtbag Ales Brewery and Taproom,micro,3623 Legion Rd,Hope Mills,North Carolina,28348-8413,United States,-78.9220969691311,34.9939752576824,9104262537,http://www.dirtbagales.com,2018-08-24T15:41:33.473Z,28348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,5273,Thirsty Monk Brewery,brewpub,92 Patton Ave,Asheville,North Carolina,28801-3315,United States,-82.5553147520814,35.59410425,8282545470,http://www.monkpub.com,2018-08-24T15:42:51.462Z,28801
331,5283,Triskelion Brewing Company,micro,340 7th Avenue East,Hendersonville,North Carolina,28792,United States,-82.4578969903109,35.3206001746298,8283887051,http://www.triskelionbrewing.com,2018-08-24T15:42:55.346Z,28792
332,5293,Valley River Brewery & Eatery,brewpub,71 Tennessee St,Murphy,North Carolina,28906-2938,United States,-84.0352081706441,35.0889040369639,8288372337,http://www.valleyriverbrewery.com,2018-08-24T15:42:59.685Z,28906
333,5304,White Street Brewing Company,micro,218 S White St,Wake Forest,North Carolina,27587-2742,United States,-78.5100617,35.9774896,9196479439,http://www.whitestreetbrewing.com,2018-08-24T15:43:05.047Z,27587


# Add to SQLite Database

In [7]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [8]:
# connect to sqlite database and add data
engine = create_engine('sqlite:///../static/data/bredat.sqlite')
subdf.to_sql('breweries', con=engine, if_exists='replace')

In [9]:
# check sqlite database
engine.execute("SELECT * FROM breweries").fetchall()[0]

(0, 5016, 'Bark Brewing Company', 'brewpub', '3021 Spring Garden St', 'Greensboro', 'North Carolina', '27403-1968', 'United States', '-79.8436043406271', '36.0620103554184', '3368561406', 'http://barkbrewingcompany.com', '2018-08-24T15:41:02.410Z', '27403')